<a href="https://colab.research.google.com/github/debu-sinha/building-ai-agents/blob/main/Basic_ReAct_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Install all dependency packages
#Remember to execute this before running any of the exercises

!pip install langchain
!pip install langchain-openai
!pip install langchain_community
!pip install langchain_chroma
!pip install langgraph
!pip install tenacity
!pip install pysqlite3-binary
!pip install pandas
!pip install pypdf
!pip install nbformat

### Setup Function Tools for ReAct Agent

In [3]:
from langchain_core.tools import tool

@tool
def find_sum(x: int, y:int) -> int :
  """
  This function takes as input two integers and returns their sum
  """
  return x+y

@tool
def find_product(x: int, y: int) -> int:
  """
  This function takes as input two integers and returns their product.
  """
  return x*y

### Create a basic ReAct Agent

In [6]:
from google.colab import userdata
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=userdata.get('oai'))

response = model.invoke("Hello, how are you?")
print(response.content)

Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to assist you with anything you need. How can I help you today?


In [8]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import SystemMessage

agent_tools = [find_sum, find_product]

#System prompt
system_prompt=SystemMessage(
    """You are a Math genius who can solve math problems. Solve the
    problems provided by the user, by using only tools available.
    Do not solve the problem yourself"""
)

agent_graph=create_react_agent(
    model,
    prompt=system_prompt,
    tools=agent_tools)

### Execute the ReAct Agent

In [10]:
#Example 1
inputs = {"messages":[("user","what is the sum of 2 and 3 ?"), ("user","what is the product of 2 and 3 ?")]}

result = agent_graph.invoke(inputs)

#Get the final answer
print(f"Agent returned : {result['messages'][-1].content} \n")

print("Step by Step execution : ")
for message in result['messages']:
    print(message.pretty_repr())


Agent returned : The sum of 2 and 3 is 5, and the product of 2 and 3 is 6. 

Step by Step execution : 
================================ Human Message =================================

what is the sum of 2 and 3 ?
================================ Human Message =================================

what is the product of 2 and 3 ?
================================== Ai Message ==================================
Tool Calls:
  find_sum (call_aTRZQg0yWyznpB4bmExl36fB)
 Call ID: call_aTRZQg0yWyznpB4bmExl36fB
  Args:
    x: 2
    y: 3
  find_product (call_IF9OqaC60gPs1gNLCEYf4IZl)
 Call ID: call_IF9OqaC60gPs1gNLCEYf4IZl
  Args:
    x: 2
    y: 3
================================= Tool Message =================================
Name: find_sum

5
================================= Tool Message =================================
Name: find_product

6
================================== Ai Message ==================================

The sum of 2 and 3 is 5, and the product of 2 and 3 is 6.


### Debugging the Agent

In [11]:
agent_graph=create_react_agent(
    model,
    prompt=system_prompt,
    tools=agent_tools,
    debug=True)

inputs = {"messages":[("user","what is the sum of 2 and 3 ?")]}

result = agent_graph.invoke(inputs)

[values] {'messages': [HumanMessage(content='what is the sum of 2 and 3 ?', additional_kwargs={}, response_metadata={}, id='754e283f-79ab-4479-840a-385eac716b37')]}
[updates] {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Yet8PvvfKbofEafsDeYWC7Hc', 'function': {'arguments': '{"x":2,"y":3}', 'name': 'find_sum'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 132, 'total_tokens': 150, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CDAVvFuHN3Heu85565JGMY3avlvE8', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--02a7bbb0-4e4a-417a-83c5-bfa207331dd6-0', tool_calls=[{'name': 'find_sum', 'args': {'x': 2, 'y'